In [ ]:
# default_exp dashboards

# dashboards

> Supplies dashboards to investigate datasets and training results. Dashboards are defined as classes, to show the dashboard use the .show() function on an dashboard instance.

In [ ]:
#export
from typing import Union

from bokeh.plotting import show, output_notebook
from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter
from bokeh.models import ColumnDataSource

import panel as pn
import panel.widgets as pnw
import pandas as pd

from icevision_dashboard.utils import *
from icevision_dashboard.components import *

In [ ]:
pn.extension()

## Test data setup

In [ ]:
import icedata

In [ ]:
test_data_dir = icedata.fridge.load_data()
test_class_map = icedata.fridge.class_map()
test_parser = icedata.fridge.parser(test_data_dir, test_class_map)
test_train_records, test_valid_records = test_parser.parse()
test_record_dataset = RecordDataset(test_valid_records, test_class_map)
test_record_dataset_no_class_map = RecordDataset(test_valid_records)

## Dataset overview

In [ ]:
#export
class RecordDatasetOverview:
    def __init__(self, record_dataset, height=500, width=1500):
        """If records is None no gallery will be generated. If the data is generated with aggregate_record_data and class map is used the class map needs to be provided as well."""
        self.record_dataset = record_dataset
        self.height = height
        self.width = width
        
    def _generate_dataset_tab(self):
        overview_table = table_from_dataframe(self.record_dataset.data, width=self.width, height=self.height)
        return pn.Column(overview_table)
        
    def _generate_image_tab(self):
        overview_table = table_from_dataframe(self.record_dataset.image_stats, width=self.width, height=50)
        if self.record_dataset.records is None:
            img_gallery = None
        else:
            img_gallery = gallery(self.record_dataset.records, self.record_dataset.class_map, height=self.height-20)
        return pn.Column(pn.Row(overview_table, align="center"), pn.Row(img_gallery, align="center"))
        
    def _generate_class_tab(self):
        overview_table = table_from_dataframe(self.record_dataset.class_stats, height=150)
        mixing_matrix_df, mapping = calculate_mixing_matrix(self.record_dataset.data, mixing_col="filepath", mixing_objects="label")
        plot_height = min(self.height-150, int(self.width/2))
        mixing_histogram = histogram_2d(mixing_matrix_df, "row_name", "col_name", "values", height=plot_height, width=plot_height)
        stacked_annotations_per_image_hist = annotations_per_image_stacked_hist(self.record_dataset.data, height=plot_height, width=plot_height)
        return pn.Column(pn.Row(overview_table, align="center"), pn.Row(mixing_histogram, stacked_annotations_per_image_hist, align="start"), align="center")
        
    def show(self):
        dataset_tab = self._generate_dataset_tab()
        image_tab = self._generate_image_tab()
        class_tab = self._generate_class_tab()
        
        return pn.Tabs(("Dataset overview", dataset_tab), ("Image stats", image_tab), ("Class stats", class_tab), width=self.width, align="start")

In [ ]:
dso = RecordDatasetOverview(test_record_dataset, height=500)
dso.show()

In [ ]:
dso = RecordDatasetOverview(test_record_dataset_no_class_map, height=700, width=500)
dso.show()

In [ ]:
#export
class MultiDatasetOverview:
    def __init__(self, datasets, height=500, width=1500):
        self._datasets = datasets
        self.width = width
        self.height = height
    
    def show(self):
        if len(self._datasets) == 0:
            return None
        del_button = pnw.Button(name="Delete", width=self.width)
        datasets_overview = create_datasets_overview(self._datasets, width=self.width, height=int(0.3*self.height))
        dataset_overview = RecordDatasetOverview(self._datasets[datasets_overview.selection[0]], width=self.width, height=int(self.height*0.7)).show()
        gui = pn.Column(del_button, datasets_overview, dataset_overview)
        
        def update_on_table_selection(selection):
            nonlocal dataset_overview
            old_active = dataset_overview.active
            dataset_overview = DatasetOverview(data=self._datasets[datasets_overview.selection[0]], width=self.width, height=int(self.height*0.7)).show()
            dataset_overview.active = old_active
            gui[-1] = dataset_overview
        datasets_overview.param.watch(update_on_table_selection, 'selection')
        
        def delete_entry(clicks):
            nonlocal datasets_overview
            nonlocal dataset_overview
            nonlocal gui
            self._datasets = [dataset for index, dataset in enumerate(self._datasets) if index not in datasets_overview.selection]
            datasets_overview = create_datasets_overview(self._datasets, width=self.width, height=int(0.3*self.height))
            datasets_overview.param.watch(update_on_table_selection, 'selection')
            gui[1] = datasets_overview
            
            if len(self._datasets) == 0:
                gui[-1] = None
            else:
                old_active = dataset_overview.active
                dataset_overview = DatasetOverview(data=self._datasets[datasets_overview.selection[0]], width=self.width, height=int(self.height*0.7)).show()
                dataset_overview.active = old_active
                gui[-1] = dataset_overview
        del_button.on_click(delete_entry)
        
        return gui

In [ ]:
mdo = MultiDatasetOverview([test_record_dataset, test_record_dataset_no_class_map], width=800, height=500)
mdo.show()

In [ ]:
mdo_empty = MultiDatasetOverview([])
assert mdo_empty.show() is None

## Dataset filter

In [ ]:
#export
class DatasetFilter:
    """Creates a gallery with filter options to select subsets of the data. If the export_varialbe is set to a variable, that is a list a dict with the current selection (pd.Dataframe (key: data) and list of records (key: records)) will be append to the list."""
    def __init__(self, record_dataset, export_variable: Union[list, None] = None, height=500, width=500):
        self.record_dataset = record_dataset
        self.export_variable = export_variable
        self.width = width
        self.height = height
    
        self.filters = []
    
    @staticmethod
    def _get_mask_from_range(values, selections):
        return (values >= selections[0]) & (values < selections[1])
    
    @staticmethod
    def export_filter(filtered_records, filtered_data):
        "Filter function applied to the output of get_filtered_records_and_data inside the show function. This is to allow for modification of the exported data."
        return {"data": filtered_data, "records": filtered_records}
    
    def show(self):
        """Creates a gallery with filter options to select subsets of the data. If the export_varialbe is set to a variable, that is a list a dict with the current selection (pd.Dataframe (key: data) and list of records (key: records)) will be append to the list."""
        # generate bbox filters
        options = self.record_dataset.data["label"].unique()
        options.sort()
        bbox_class_filter = pnw.MultiSelect(name="Class", options=options.tolist(), value=options.tolist())
        bbox_num_annotations_filter = generate_range_filter(self.record_dataset.data["num_annotations"], "Num. Annotations", steps=self.record_dataset.data["num_annotations"].max()+3, width=int(self.width/2))
        bbox_area_filter = generate_range_filter(self.record_dataset.data["area"], "Area", steps=50, width=int(self.width/2))
        bbox_width_filter = generate_range_filter(self.record_dataset.data["bbox_width"], "Bbox width", steps=50, width=int(self.width/2))
        bbox_height_filter = generate_range_filter(self.record_dataset.data["bbox_height"], "Bbbox height", steps=50, width=int(self.width/2))
        bbox_ratio_filter = generate_range_filter(self.record_dataset.data["bbox_ratio"], "Ratio", steps=50, width=int(self.width/2))
        annotation_filters = pn.Row(pn.Column(bbox_class_filter, bbox_num_annotations_filter, bbox_area_filter, width=int(self.width/2)), pn.Column(bbox_width_filter, bbox_height_filter, bbox_ratio_filter, width=int(self.width/2)))

        # generate image filters
        file_creation_modification_time_filter = generate_creation_modification_time_filter(self.record_dataset.data, width=int(self.width/2))
        file_width_filter = generate_range_filter(self.record_dataset.data["width"], "Width", steps=50, width=int(self.width/2))
        file_height_filter = generate_range_filter(self.record_dataset.data["height"], "Height", steps=50, width=int(self.width/2))
        file_filters = pn.Row(pn.Column(file_creation_modification_time_filter, width=int(self.width/2)), pn.Column(file_width_filter, file_height_filter, width=int(self.width/2)))

        filters = pn.Tabs(("File filters", file_filters), ("Annotation filters", annotation_filters), ("Gallery", None), width=self.width, active=1)
        
        def get_filtered_records_and_data():
            filtered_data = self.record_dataset.data[
                self.record_dataset.data["label"].isin(bbox_class_filter.value)
                & self._get_mask_from_range(self.record_dataset.data["num_annotations"], bbox_num_annotations_filter[0].value)
                & self._get_mask_from_range(self.record_dataset.data["area"], bbox_area_filter[0].value)
                & self._get_mask_from_range(self.record_dataset.data["bbox_width"], bbox_width_filter[0].value)
                & self._get_mask_from_range(self.record_dataset.data["bbox_height"], bbox_height_filter[0].value)
                & self._get_mask_from_range(self.record_dataset.data["bbox_ratio"], bbox_ratio_filter[0].value)
                & self._get_mask_from_range(self.record_dataset.data["creation_date"], file_creation_modification_time_filter[1].value)
                & self._get_mask_from_range(self.record_dataset.data["modification_date"], file_creation_modification_time_filter[2].value)
                & self._get_mask_from_range(self.record_dataset.data["width"], file_width_filter[0].value)
                & self._get_mask_from_range(self.record_dataset.data["height"], file_height_filter[0].value)
            ]
            filtered_records = [self.record_dataset.records[i] for i in filtered_data["record_index"].unique()]
            return filtered_records, filtered_data

        @pn.depends(
            bbox_class_filter.param.value, bbox_num_annotations_filter[0].param.value_throttled, bbox_area_filter[0].param.value_throttled, 
            bbox_width_filter[0].param.value_throttled, bbox_height_filter[0].param.value_throttled, bbox_ratio_filter[0].param.value_throttled,
            file_creation_modification_time_filter[1].param.value_throttled, file_creation_modification_time_filter[2].param.value_throttled, 
            file_width_filter[0].param.value_throttled, file_height_filter[0].param.value_throttled
        )
        def _gallery(
            classes_selection, num_annotation_selection, area_selection, width_selection, height_selection, ratio_selection, 
            image_creation_date_selection, image_modification_date_selection, image_with_selection, image_height_selection
        ):
            filtered_records, filtered_data = get_filtered_records_and_data()

            # update filter histograms
            bbox_num_annotations_filter[1] = histogram(
                self.record_dataset.data.groupby("id").count()["width"], bins=20, height=100, width=int(self.width/2), 
                range=(self.record_dataset.data.groupby("id").count()["width"].min(), self.record_dataset.data.groupby("id").count()["width"].max()), remove_tools=True)
            
            bbox_area_filter[1] = histogram(
                filtered_data["area"], bins=20, height=100, width=int(self.width/2), 
                range=(self.record_dataset.data["area"].min(), self.record_dataset.data["area"].max()), remove_tools=True)
            
            bbox_width_filter[1] = histogram(
                filtered_data["bbox_width"], bins=20, height=100, width=int(self.width/2), 
                range=(self.record_dataset.data["bbox_width"].min(), self.record_dataset.data["bbox_width"].max()), remove_tools=True)
            
            bbox_height_filter[1] = histogram(
                filtered_data["bbox_height"], bins=20, height=100, width=int(self.width/2), 
                range=(self.record_dataset.data["bbox_height"].min(), self.record_dataset.data["bbox_height"].max()), remove_tools=True)
            
            bbox_ratio_filter[1] = histogram(
                filtered_data["bbox_ratio"], bins=20, height=100, width=int(self.width/2), 
                range=(self.record_dataset.data["bbox_ratio"].min(), self.record_dataset.data["bbox_ratio"].max()), remove_tools=True)

            if len(filtered_records) == 0:
                return None
            else:
                return gallery(filtered_records, width=self.width, height=self.height)

        if self.export_variable is not None:
            def export(event):
                filtered_records, filtered_data = get_filtered_records_and_data()
                export_result = self.export_filter(filtered_records, filtered_data)
                self.export_variable.append(export_result)

            export_button = pnw.Button(name="Export")           
            export_button.on_click(export)
            filters[2] = ("Gallery", _gallery)
            return pn.Column(filters, export_button, align="center")
        else:
            return pn.Column(filters, align="center")

In [ ]:
dump_var = []
ds_filter = DatasetFilter(test_record_dataset, export_variable=dump_var, width=1000, height=500).show()
# trigger click event to test if record is added to dump_var
ds_filter[-1].clicks += 1
assert len(dump_var) == 1
assert len(dump_var[0]["records"]) == len(test_valid_records)
ds_filter

## Dataset creator

In [ ]:
#export
class DatasetFilterThatOnlyExportsStats(DatasetFilter):
    @staticmethod
    def export_filter(filtered_records, filtered_data):
        return filtered_data

In [ ]:
dump_var = []
dstoes_filter = DatasetFilterThatOnlyExportsStats(test_record_dataset, export_variable=dump_var, height=400).show()
# trigger click event to test if record is added to dump_var
dstoes_filter[-1].clicks += 1
assert len(dump_var) == 1
assert isinstance(dump_var[0], pd.DataFrame)
dstoes_filter

In [ ]:
#export
class DatasetCreator:
    def __init__(self, record_dataset, class_map=None, height=500, width=1000):
        self.height = height
        self.width = width
        self.record_dataset = record_dataset
        self._datasets = ObservableList([])
        self._datasets.register_callback(self.update_datasets_overview)
        self._filter = DatasetFilterThatOnlyExportsStats(self.record_dataset, export_variable=self._datasets, width=self.width, height=self.height).show()
        self._datasets_overview = MultiDatasetOverview(self._datasets.observable_list, width=self.width, height=self.height)
        print(self._datasets_overview._datasets)
        self._gui = pn.Tabs(("Filter datasets", self._filter), ("Inspect datasets", self._datasets_overview.show()), ("Export datasets", self.export_gui()))
        
    @staticmethod
    def export_datasets(datasets):
        pass
        
    def export_gui(self):
        export_path = pnw.TextInput(name="Export path", value="./datasets")
        export_button = pnw.Button(name="Export")
        
        datasets_overview = create_datasets_overview(self._datasets)
#         def create_datasets_overview(datasets, width=500, height=100):
#             "Creates an overview table for a list of datasets were each datasets is a dataframe."
#             dataset_stats_list = []
#             for dataset in datasets:
#                 dataset_stats = calculate_dataset_stats(dataset)
#                 dataset_stats["classes"] = ", ".join([str(entry) for entry in dataset_stats["classes"]])
#                 dataset_stats_list.append(dataset_stats)
#             dataset_stats_df = pd.DataFrame(dataset_stats_list)

#             template = """<span href="#" data-toggle="tooltip" title="<%= value %>"><%= value %></span>"""
#             table = pnw.DataFrame(dataset_stats_df, formatters={"classes": HTMLTemplateFormatter(template=template)}, selection=[0], width=width, height=height)
#             return table
        
        
        return pn.Column(export_path, datasets_overview, export_button)
        
    def update_datasets_overview(self, datasets):
        print(self._datasets_overview.show())
#         self._gui[1] = ("Inspect datasets", self._datasets_overview.show())
#         self._gui[-1] = ("Export datasets", self.export_gui())
        pass
        
    def show(self):
        return self._gui

In [ ]:
dsc = DatasetCreator(test_record_dataset, test_class_map)
dsc.show()

In [ ]:
dsc._datasets.append(1)

In [ ]:
create_datasets_overview([test_record_dataset])